In [71]:
from lithops import Storage,storage
from lithops.multiprocessing import Pool
from itertools import product
import lithops
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json
import io
from ast import literal_eval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from mpl_toolkits.basemap import Basemap

In [2]:
stg = Storage()
data = stg.get_object('analysis.data', 'processeddata/sentiments.csv').decode()
data = io.StringIO(data)
df = pd.read_csv(data, sep=",")
print(df)

                        Id        Date  Retweet           Location  Sentiment
0      1220910820378337280  2020-01-25      1.0       Vegas Strong     0.4939
1      1221750876538396672  2020-01-27      1.0    Warszawa Polska     0.0000
2      1221831050961145859  2020-01-27      1.0      hiroshima JPN     0.3182
3      1221836638084456448  2020-01-27      1.0                NaN    -0.5994
4      1221839402164293632  2020-01-27      1.0      United States     0.0000
...                    ...         ...      ...                ...        ...
75102  1393758103720562689  2021-05-16      1.0                NaN    -0.8225
75103  1393763502058217473  2021-05-16      1.0                NaN    -0.7269
75104  1393768387071778822  2021-05-16      1.0                NaN    -0.4019
75105  1393773076286869504  2021-05-16      1.0  God's own country     0.0000
75106  1393777586602450945  2021-05-16      1.0                NaN     0.0000

[75107 rows x 5 columns]


In [64]:
def average(date, df):
    mean = df.loc[df['Date'] == date]['Sentiment'].mean()
    return(mean)

In [53]:
print(average('2020-03-04'))

0.002894117647058819


In [70]:
dates = sorted(set(df['Date']))



In [69]:
with Pool() as p:
    r = p.starmap(average,)
print(r)

2021-05-26 17:20:39,273 [INFO] lithops.config -- Lithops v2.3.3
2021-05-26 17:20:39,298 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-gb
2021-05-26 17:20:39,299 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: us-south - Namespace: david.gaseni@estudiants.urv.cat_dev
2021-05-26 17:20:39,300 [INFO] lithops.executors -- Serverless Executor created with ID: 133891-6
2021-05-26 17:20:39,301 [INFO] lithops.invokers -- ExecutorID 133891-6 | JobID A000 - Selected Runtime: buzzerage/sd_fullpackages:latest - 256MB
/usr/lib/python3.9/inspect.py:351: FutureWarning: _AXIS_NAMES has been deprecated.
  value = getattr(object, key)
/usr/lib/python3.9/inspect.py:351: FutureWarning: _AXIS_NUMBERS has been deprecated.
  value = getattr(object, key)


AttributeError: module 'numpy.ma' has no attribute 'mrecords'